In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

#  SETTINGS
MODEL_PATH = "plant_disease_resnet50.h5"
IMAGE_SIZE = (224, 224)

# Load the trained model
model = tf.keras.models.load_model(MODEL_PATH)

# Disease-to-treatment dictionary (sample mapping, extend as needed)
treatment_dict = {
    "Apple___Black_rot": "Remove infected leaves/fruit, apply fungicide (e.g., Captan).",
    "Apple___healthy": "No treatment needed. Keep monitoring the plant.",
    "Corn___Common_rust": "Apply fungicide (e.g., azoxystrobin), use resistant hybrids.",
    "Corn___healthy": "No treatment needed.",
    "Potato___Early_blight": "Use chlorothalonil-based fungicides, remove debris.",
    "Potato___Late_blight": "Apply fungicides with mancozeb or metalaxyl, destroy infected plants.",
    "Potato___healthy": "No treatment required.",
    "Tomato___Leaf_Mold": "Improve ventilation, apply copper-based fungicides.",
    "Tomato___healthy": "No treatment needed.",
    # more classes as per dataset
}

# SEVERITY ESTIMATION FUNCTION
def estimate_severity(probability):
    if probability < 0.60:
        return "Mild"
    elif probability < 0.85:
        return "Moderate"
    else:
        return "Severe"

# PREDICTION FUNCTION
def predict_disease(img_path, class_indices):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Prediction
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds, axis=1)[0]
    confidence = np.max(preds)

    # Map index to class name
    class_labels = {v: k for k, v in class_indices.items()}
    disease_name = class_labels[predicted_class]

    # Severity estimation
    severity = estimate_severity(confidence)

    # Treatment lookup
    treatment = treatment_dict.get(disease_name, "No treatment info available. Consult local guidelines.")

    # Output
    result = f"Disease: {disease_name} | Severity: {severity} | Treatment: {treatment}"
    return result


class_indices = {
    0: "Apple___Apple_scab",
    1: "Apple___Black_rot",
    2: "Apple___Cedar_apple_rust",
    3: "Apple___healthy",
    4: "Blueberry___healthy",
    5: "Cherry_(including_sour)___Powdery_mildew",
    6: "Cherry_(including_sour)___healthy",
    7: "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
    8: "Corn_(maize)___Common_rust_",
    9: "Corn_(maize)___Northern_Leaf_Blight",
    10: "Corn_(maize)___healthy",
    11: "Grape___Black_rot",
    12: "Grape___Esca_(Black_Measles)",
    13: "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    14: "Grape___healthy",
    15: "Orange___Haunglongbing_(Citrus_greening)",
    16: "Peach___Bacterial_spot",
    17: "Peach___healthy",
    18: "Pepper,_bell___Bacterial_spot",
    19: "Pepper,_bell___healthy",
    20: "Potato___Early_blight",
    21: "Potato___Late_blight",
    22: "Potato___healthy",
    23: "Raspberry___healthy",
    24: "Soybean___healthy",
    25: "Squash___Powdery_mildew",
    26: "Strawberry___Leaf_scorch",
    27: "Strawberry___healthy",
    28: "Tomato___Bacterial_spot",
    29: "Tomato___Early_blight",
    30: "Tomato___Late_blight",
    31: "Tomato___Leaf_Mold",
    32: "Tomato___Septoria_leaf_spot",
    33: "Tomato___Spider_mites Two-spotted_spider_mite",
    34: "Tomato___Target_Spot",
    35: "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    36: "Tomato___Tomato_mosaic_virus",
    37: "Tomato___healthy"
}


test_image_path = "test_leaf.jpg"   # test image
if os.path.exists(test_image_path):
    print(predict_disease(test_image_path, class_indices))
else:
    print("⚠️ Please provide a valid test image path.")



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Disease: Powdery Mildew, Infection Level: Mild to Severe, Treatment: Apply fungicide; 2 ml per liter of water


In [1]:
import kagglehub

path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'new-plant-diseases-dataset' dataset.
Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [2]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip -d dataset

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open plantdisease.zip, plantdisease.zip.zip or plantdisease.zip.ZIP.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import os

# SETTINGS

DATASET_DIR = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"   # dataset folder with subfolders for each class
IMAGE_SIZE = (224, 224)   # ResNet50 input size
BATCH_SIZE = 32
EPOCHS = 10
MODEL_SAVE_PATH = "plant_disease_resnet50.h5"


# DATA LOADING

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
print("Classes:", train_generator.class_indices)


#MODEL CREATION (ResNet50)

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


# COMPILE

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# TRAIN

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)


# SAVE MODEL

model.save(MODEL_SAVE_PATH)
print(f"Model saved at {MODEL_SAVE_PATH}")


Found 56251 images belonging to 38 classes.
Found 14044 images belonging to 38 classes.
Classes: {'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
   3/1758 ━━━━━━━━━━━━━━━━━━━━ 2:39:13 5s/step - accuracy: 0.0191 - loss: 4.3274

KeyboardInterrupt: 